do pliku `AndroidManifest.xml`
```xml
    <uses-permission android:name="android.permission.INTERNET" />

    <application
        android:name=".RandomUserApplication"
    ...
    </application>
```

do pliku `build.gradle(Project)`
```kotlin
plugins {
    alias(libs.plugins.android.application) apply false
    alias(libs.plugins.kotlin.android) apply false
    alias(libs.plugins.kotlin.compose) apply false
    id("com.google.dagger.hilt.android") version "2.57.2" apply false
    id("com.google.devtools.ksp") version "2.0.21-1.0.27" apply false
    id("androidx.room") version "2.8.1" apply false
}
```

do pliku `build.gradle(Module)`
```kotlin
plugins {
    ...
    id("com.google.devtools.ksp")
    id("com.google.dagger.hilt.android")
    id("androidx.room")
}

android {
    ...
    room {
        schemaDirectory("$projectDir/schemas")
    }
}

dependencies {

    implementation("com.google.dagger:hilt-android:2.57.2")
    ksp("com.google.dagger:hilt-android-compiler:2.57.2")

    // ViewModel i Lifecycle w Compose
    implementation("androidx.lifecycle:lifecycle-viewmodel-compose:2.9.4")
    implementation("androidx.lifecycle:lifecycle-runtime-compose:2.9.4")
    implementation("androidx.hilt:hilt-navigation-compose:1.3.0")

    implementation("androidx.room:room-runtime:2.8.1")
    ksp("androidx.room:room-compiler:2.8.1")
    implementation("androidx.room:room-ktx:2.8.1")

    // Retrofit do komunikacji sieciowej
    implementation("com.squareup.retrofit2:retrofit:3.0.0")
    // Konwerter GSON do parsowania JSON
    implementation("com.squareup.retrofit2:converter-gson:3.0.0")
    ...
}
```

```kotlin

// =================================================================================
// --- 1. Konfiguracja Hilt ---
// =================================================================================
@HiltAndroidApp
class MovieApplication : Application()

// =================================================================================
// --- 2. Warstwa Danych ---
// =================================================================================

// --- Modele DTO ---
data class MovieListResponse(val results: List<MovieDto>)
data class MovieDto(
    val id: Int,
    val title: String,
    val overview: String
)

// --- Encje Room ---
@Entity(tableName = "movies_cache")
data class MovieEntity(
    @PrimaryKey val id: Int,
    val title: String,
    val overview: String
)

@Entity(tableName = "favorite_movie_ids")
data class FavoriteMovieEntity(@PrimaryKey val id: Int)

// --- Obiekt do odczytu z bazy (wynik JOIN) ---
data class Movie(
    val id: Int,
    val title: String,
    val overview: String,
    val isFavorite: Boolean
)

// --- Retrofit API Service ---
interface TmdbApiService {
    @GET("movie/popular")
    suspend fun getPopularMovies(
        @RetrofitQuery("api_key") apiKey: String,
        @RetrofitQuery("language") language: String = "pl-PL"
    ): MovieListResponse
}

// --- Room DAO ---
@Dao
interface MovieDao {
    @Transaction
    @Query("""
        SELECT 
            movies_cache.*, 
            (favorite_movie_ids.id IS NOT NULL) as isFavorite
        FROM movies_cache
        LEFT JOIN favorite_movie_ids ON movies_cache.id = favorite_movie_ids.id
        ORDER BY title ASC
    """)
    fun getMoviesWithFavoriteStatus(): Flow<List<Movie>>

    @Upsert
    suspend fun upsertMovies(movies: List<MovieEntity>)

    @Query("DELETE FROM movies_cache")
    suspend fun clearMoviesCache()

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    suspend fun addToFavorites(favorite: FavoriteMovieEntity)

    @Query("DELETE FROM favorite_movie_ids WHERE id = :movieId")
    suspend fun removeFromFavorites(movieId: Int)

}

@Database(entities = [MovieEntity::class, FavoriteMovieEntity::class], version = 1, exportSchema = false)
abstract class AppDatabase : RoomDatabase() {
    abstract fun movieDao(): MovieDao
}

// --- Repozytorium ---
class MovieRepository @Inject constructor(
    private val apiService: TmdbApiService,
    private val movieDao: MovieDao
) {
    private val apiKey = "TUTAJ_WSTAW_SWOJ_KLUCZ_API"

    private val mutex = Mutex()

    val moviesStream: Flow<List<Movie>> = movieDao.getMoviesWithFavoriteStatus()

    suspend fun toggleFavoriteStatus(movie: Movie) {
        mutex.withLock {
            if (movie.isFavorite) {
                movieDao.removeFromFavorites(movie.id)
            } else {
                movieDao.addToFavorites(FavoriteMovieEntity(id = movie.id))
            }
        }
    }

    suspend fun refreshMovies() {
        mutex.withLock {
            try {
                val response = apiService.getPopularMovies(apiKey)
                val entities = response.results.map { dto ->
                    MovieEntity(
                        id = dto.id,
                        title = dto.title,
                        overview = dto.overview
                    )
                }

                movieDao.upsertMovies(entities)
            } catch (e: Exception) {
                Log.e("MovieRepository", "Failed to fetch movies", e)
                throw e
            }
        }
    }
}

// =================================================================================
// --- 3. Moduł Hilt ---
// =================================================================================
@Module
@InstallIn(SingletonComponent::class)
object AppModule {
    @Provides
    @Singleton
    fun provideAppDatabase(@ApplicationContext context: Context): AppDatabase =
        Room.databaseBuilder(context, AppDatabase::class.java, "movie_database").build()

    @Provides
    @Singleton
    fun provideMovieDao(database: AppDatabase): MovieDao = database.movieDao()

    @Provides
    @Singleton
    fun provideRetrofit(): Retrofit = Retrofit.Builder()
        .baseUrl("https://api.themoviedb.org/3/")
        .addConverterFactory(GsonConverterFactory.create())
        .build()

    @Provides
    @Singleton
    fun provideApiService(retrofit: Retrofit): TmdbApiService = retrofit.create(TmdbApiService::class.java)
}

// =================================================================================
// --- 4. Architektura: ViewModel i Stan ---
// =================================================================================
data class MoviesUiState(
    val movies: List<Movie> = emptyList(),
    val isLoading: Boolean = false,
    val error: String? = null
)

@HiltViewModel
class MoviesViewModel @Inject constructor(
    private val repository: MovieRepository
) : ViewModel() {
    private val _isLoading = MutableStateFlow(false)
    private val _error = MutableStateFlow<String?>(null)

    val uiState: StateFlow<MoviesUiState> = combine(
        repository.moviesStream, _isLoading, _error
    ) { movies, isLoading, error ->
        MoviesUiState(movies, isLoading, error)
    }.stateIn(
        scope = viewModelScope,
        started = SharingStarted.WhileSubscribed(5000),
        initialValue = MoviesUiState(isLoading = true)
    )

    init {
        refresh()
    }

    fun refresh() {
        viewModelScope.launch {
            _isLoading.value = true
            _error.value = null
            try {
                repository.refreshMovies()
            } catch (e: Exception) {
                _error.value = "Nie udało się odświeżyć danych."
            } finally {
                _isLoading.value = false
            }
        }
    }

    fun onFavoriteClicked(movie: Movie) {
        viewModelScope.launch {
            repository.toggleFavoriteStatus(movie)
        }
    }
}

// =================================================================================
// --- 5. UI: Ekrany ---
// =================================================================================

@AndroidEntryPoint
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContent {
            MaterialTheme {
                MovieListScreen()
            }
        }
    }
}

@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun MovieListScreen(viewModel: MoviesViewModel = hiltViewModel()) {
    val uiState by viewModel.uiState.collectAsStateWithLifecycle()

    Scaffold(
        topBar = {
            TopAppBar(
                title = { Text("Popularne Filmy (Offline)") },
                actions = {
                    IconButton(onClick = viewModel::refresh, enabled = !uiState.isLoading) {
                        Icon(Icons.Default.Refresh, contentDescription = "Odśwież")
                    }
                }
            )
        }
    ) { padding ->
        Box(modifier = Modifier.fillMaxSize().padding(padding)) {
            if (uiState.isLoading && uiState.movies.isEmpty()) {
                CircularProgressIndicator(modifier = Modifier.align(Alignment.Center))
            } else if (uiState.error != null && uiState.movies.isEmpty()) {
                Text(
                    text = uiState.error!!,
                    color = MaterialTheme.colorScheme.error,
                    modifier = Modifier.align(Alignment.Center).padding(16.dp)
                )
            } else {
                LazyColumn(modifier = Modifier.fillMaxSize(), contentPadding = PaddingValues(16.dp)) {
                    items(uiState.movies, key = { it.id }) { movie ->
                        MovieItem(
                            movie = movie,
                            onFavoriteClick = { viewModel.onFavoriteClicked(movie) }
                        )
                        Spacer(modifier = Modifier.height(12.dp))
                    }
                }
            }
            if (uiState.isLoading) {
                LinearProgressIndicator(modifier = Modifier.fillMaxWidth())
            }
        }
    }
}

@Composable
fun MovieItem(movie: Movie, onFavoriteClick: () -> Unit) {
    Card(elevation = CardDefaults.cardElevation(defaultElevation = 2.dp)) {
        Row(
            modifier = Modifier.fillMaxWidth().padding(16.dp),
            verticalAlignment = Alignment.CenterVertically
        ) {
            Column(modifier = Modifier.weight(1f)) {
                Text(text = movie.title, style = MaterialTheme.typography.titleMedium, fontWeight = FontWeight.Bold)
                Spacer(modifier = Modifier.height(4.dp))
                Text(
                    text = movie.overview,
                    style = MaterialTheme.typography.bodySmall,
                    maxLines = 4
                )
            }
            IconButton(onClick = onFavoriteClick) {
                Icon(
                    imageVector = if (movie.isFavorite) Icons.Filled.Favorite else Icons.Filled.FavoriteBorder,
                    contentDescription = "Ulubione",
                    tint = if (movie.isFavorite) Color.Red else Color.Gray
                )
            }
        }
    }
}

```